<a href="https://colab.research.google.com/github/Cinex10/faq_extractor/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scikit-learn

In [3]:
!pip install -q hdbscan

## Importing libraries

In [4]:
from sklearn.manifold import TSNE
#from sklearn.cluster import DBSCAN,HDBSCAN
from hdbscan import HDBSCAN
from transformers import AutoTokenizer,DistilBertModel, DataCollatorWithPadding
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral11,Sunset3

2024-03-24 23:25:01.732413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 23:25:01.732523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 23:25:01.872117: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# NLP Phase

In [6]:
!cat data.csv | head -5

"questions"
"What's the cost of a standard room?"
"Can you provide information on your room types?"
"What are your hotel's check-in and check-out timings?"
"Are there any extended stay discounts available?"


## Loading data

In [5]:
%%writefile data.csv
"questions"
"What's the cost of a standard room?"
"Can you provide information on your room types?"
"What are your hotel's check-in and check-out timings?"
"Are there any extended stay discounts available?"
"Does the room rate include complimentary breakfast?"
"Is it permissible to have pets in the hotel?"
"Is there a fitness facility or gym within the hotel premises?"
"How do I request a late check-out?"
"Is there an airport shuttle service available?"
"Do you offer rooms with separate smoking and non-smoking options?"
"May I reserve a room with a scenic view?"
"Could you detail the room service options and operational hours?"
"Are there recommended nearby restaurants and attractions?"
"Does the room rate include Wi-Fi access and what's the network name?"
"Could you explain the hotel's reservation cancellation policy?"
"Do you offer any special packages for events or celebrations?"
"Can I request an additional bed or crib for my room?"
"What's the distance from the hotel to the closest train station?"
"Are there rooms designed for guests with mobility challenges?"
"Is there a pool or spa on the hotel premises?"
"How can I arrange transportation to local attractions?"
"What's the policy for holding luggage before check-in or after check-out?"
"Can I choose my room preferences when I book?"
"Do you offer discounts for senior citizens or AARP members?"
"Is there a concierge desk to assist with local recommendations?"
"What's the process for addressing lost or left-behind items?"
"Are there any in-room amenities like a mini-fridge or microwave?"
"What's your policy on early check-in requests?"
"Can I get assistance with tour bookings or tickets?"
"How often are the rooms cleaned and serviced?"
"Is there a dress code for the hotel's dining areas?"
"Are there any on-site meeting or event facilities?"
"Can I book a room with a balcony or terrace?"
"Is there a quiet time policy in the hotel?"
"Can you provide recommendations for nearby shopping areas?"
"Do you offer laundry or dry-cleaning services?"
"Is room keycard access required for specific hotel areas?"
"Are there designated smoking areas on the property?"
"Can I arrange for a wake-up call in the morning?"
"What's the hotel's policy on lost and found items?"
"Do you provide discounts for military personnel or veterans?"
"Are there any outdoor seating or dining options on-site?"
"Can I request a specific floor or room location?"
"How can I contact the front desk for assistance?"
"Do you offer transportation services to the city center?"
"What's the process for extending my stay?"
"Can I pre-order amenities like flowers or champagne for special occasions?"
"Are there any security measures in place within the hotel?"
"Can I leave my luggage at the front desk before check-in?"
"Do you offer babysitting or childcare services?"
"Is there a business center with printing and copying facilities?"
"How is the hotel handling COVID-19 safety measures?"
"What's the policy on lost or misplaced room keys?"
"Are there any hidden fees not included in the room rate?"
"How can I book a room for a large group or event?"
"Do you provide airport pick-up services?"
"Can I access the hotel's fitness center 24/7?"
"What's the procedure for early check-out?"
"Is there a curfew or quiet hours in the hotel?"
"Are there in-room dining options available?"
"Do you provide transportation to local hospitals or medical facilities?"
"Can I store valuable items in a hotel safe?"
"Are there any nearby scenic walking or jogging trails?"
"What's the policy for receiving mail or packages?"
"Is there a loyalty program for frequent guests?"
"Do you have a lost and found department for guests?"
"Are there any in-room entertainment options?"
"Can I change my room type upon arrival if available?"
"Are there designated pet-friendly rooms?"
"How can I access the hotel's guest reviews?"
"Is there a courtesy shuttle to popular tourist attractions?"
"Can I arrange for early morning coffee or tea delivery?"
"Do you offer assistance for planning proposals or special occasions?"
"Are there any electric vehicle charging stations on-site?"
"Can I have flowers or gifts delivered to my room?"
"What's your policy on accommodating dietary restrictions for meals?"
"Can I use a hotel phone for local calls?"
"Are there discounts for government employees or diplomats?"
"Is there a library or reading room within the hotel?"
"How can I access the hotel's event calendar?"
"Can I upgrade my room after checking in?"
"Do you provide special amenities for honeymooning couples?"
"Are there any cultural or artistic events in the area?"
"How do I access the hotel's spa or wellness center?"
"Are there any nearby scenic walking or jogging trails?"
"Can I book a room for an extended period such as a month?"
"Is there a designated area for picnics or outdoor dining?"
"How do I access room service menus and pricing?"
"Are there options for in-room cooking or kitchenettes?"
"Can I request a late-night check-in or early check-out?"
"Is there a security escort service for late-night arrivals?"
"How can I receive notifications about hotel promotions?"
"Can I participate in a hotel loyalty program upon check-in?"
"Are there options for in-room massages or spa treatments?"
"Can I get assistance with planning local sightseeing tours?"
"How can I arrange a private dining experience within the hotel?"
"Are there designated meeting or event coordinators on staff?"
"Is there a quiet area for reading or studying within the hotel?"
"Can I schedule wake-up calls with specific music or sounds?"
"Are there options for in-room cooking or kitchenettes?"
"How can I access the hotel's full menu of services and amenities?"
"Can I book a conference room or business meeting space?"
"Is there a car rental service available at the hotel?"
"Can I have special dietary preferences noted for dining?"
"How can I access the hotel's digital concierge services?"
"Are there options for arranging surprise gifts or decorations?"
"Is there a helipad or space for helicopter arrivals?"
"Can I request a designated smoking balcony or terrace?"
"How can I access the hotel's art or sculpture gallery?"
"Are there options for arranging live music or entertainment?"
"Can I receive information on hotel sustainability efforts?"
"Is there a designated area for meditation or yoga?"
"How can I access the hotel's library of books or movies?"
"Can I schedule room cleanings at specific times?"
"Are there options for in-room cooking lessons or classes?"
"How can I access the hotel's cultural or historical exhibits?"
"Can I have a private event with personal catering?"
"Is there a photography service for special occasions?"
"How can I access the hotel's VIP or concierge lounge?"
"Are there options for in-room babysitting or childcare?"
"Can I receive detailed maps of local walking tours?"
"Is there a shuttle to nearby golf courses?"
"How can I access information on hotel awards and accolades?"
"Can I have a personal trainer for in-room workouts?"
"Is there a designated area for art or craft workshops?"
"How can I access information on hotel partnerships or affiliations?"
"Can I schedule in-room laundry or dry-cleaning services?"
"Are there options for in-room language or cooking lessons?"
"Can I access the hotel's list of recommended local artists?"
"How can I receive a copy of the hotel's sustainability report?"


Overwriting data.csv


In [7]:
class QuestionsDataset(Dataset):
    def __init__(self,csv_file,transform=None):
        self.data = pd.read_csv(csv_file,index_col=None)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        sample = self.data.loc[idx]['questions']
        if self.transform:
            sample = self.transform(sample)
        return sample

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
csv_file = 'data.csv'
dataset = QuestionsDataset(csv_file,transform=tokenizer)

In [87]:
qst = pd.read_csv(csv_file)

In [11]:
dataset[0]

{'input_ids': [101, 2054, 1005, 1055, 1996, 3465, 1997, 1037, 3115, 2282, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Calculating sentences embedings

In [12]:
model = DistilBertModel.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [13]:
from torch.utils.data import DataLoader

In [14]:
datacollator = DataCollatorWithPadding(tokenizer)

In [15]:
#dataset = TensorDataset(dataset)
loader = DataLoader(dataset, batch_size=32, shuffle=False,collate_fn=datacollator)

In [16]:
for batch in loader:
    print({k:v.shape for k,v in batch.items()})

{'input_ids': torch.Size([32, 20]), 'attention_mask': torch.Size([32, 20])}
{'input_ids': torch.Size([32, 16]), 'attention_mask': torch.Size([32, 16])}
{'input_ids': torch.Size([32, 18]), 'attention_mask': torch.Size([32, 18])}
{'input_ids': torch.Size([32, 17]), 'attention_mask': torch.Size([32, 17])}
{'input_ids': torch.Size([2, 16]), 'attention_mask': torch.Size([2, 16])}


In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

cuda


In [18]:
embedings = []

for batch in loader:
    batch = {k:v.to(device) for k,v in batch.items()}
    output = model(**batch)
    embd = output.last_hidden_state[:,0]
    embedings.append(embd)

In [19]:
embedings = torch.cat(embedings,dim=0)

In [20]:
embedings.shape

torch.Size([130, 768])

## Similary matrix

In [21]:
similarity_matrix = torch.nn.functional.cosine_similarity(embedings.unsqueeze(1),embedings.unsqueeze(0),dim=2).to('cpu').detach()

In [22]:
similarity_matrix.device

device(type='cpu')

In [23]:
# Dimensionality reducing into 2 components
tsne = TSNE(n_components=2,perplexity=30,random_state=0)
reduced_vectors = tsne.fit_transform(similarity_matrix)

## Visualize similarities

In [24]:
colors = plt.cm.rainbow(np.linspace(0, 1, len(dataset)))

source = ColumnDataSource(data={
    'x': reduced_vectors[:, 0],
    'y': reduced_vectors[:, 1],
    'labels': qst,
    'colors': colors,
})

# Create a Bokeh figure
p = figure(width=1800, height=600, title="Question Similarity Visualization")

# Add a scatter plot with tooltips
scatter = p.circle('x', 'y', size=15, source=source, fill_color="colors", line_color="colors")
# Define the tooltip style with HTML/CSS attributes
tooltips = """
Question:

@labels

"""

# Add a hover tool with the defined tooltips
hover = HoverTool(renderers=[scatter], tooltips=tooltips)
p.add_tools(hover)

# Enable notebook display mode
output_notebook()

# Show the plot
show(p)

Loading BokehJS ...

# Clustering Phase

## HDBSCAN

In [40]:
dbscan = HDBSCAN(min_samples=1,metric="precomputed",min_cluster_size=2)  # "precomputed" for using a precomputed similarity matrix

In [41]:
dissimilarity_matrix = 1 - similarity_matrix
dissimilarity_matrix[dissimilarity_matrix < 0] = 0
labels = dbscan.fit_predict(dissimilarity_matrix.numpy().astype(np.double))  # Convert similarity to dissimilarity (1 - similarity) for DBSCAN

In [42]:
labels

array([-1,  6, -1, -1, -1, -1,  5, 21, 17, -1, -1,  6, -1, -1, 12,  8, -1,
       -1, -1,  5, -1, -1, 19,  7, 13, -1, -1,  3, 19, -1, -1, 15, -1, 16,
       -1,  2, -1,  0, 19,  3,  7, 15, 20, -1,  4, -1, -1, -1, -1,  2, -1,
       -1,  3, -1, -1,  4, -1,  3, 16, 14,  2, 19,  0,  3, 16, 12, 14, 19,
       -1, 22, 17, 19,  8, -1, 19,  3, 19, -1,  5, 22, 19,  8, -1, -1,  0,
       19,  0, -1, 10, 19, 13, 21, -1,  9, 19, -1, -1,  5, 20, 10, 23, 19,
       16, -1, 23,  1, -1, -1, 23,  1, 18,  0, 23, 20, 11, -1, 19, -1, 23,
        9, 19, 17, 18, 19,  0, -1, -1, 11, -1, -1])

In [110]:
def get_clusters_content(labels):
  clusters = {}

  for index,label in enumerate(labels):
    if label == -1:
      continue
    question = qst.iloc[index]['questions']
    cluster_key = f'cluster_{label}'
    data = clusters.get(cluster_key)
    # print(f'index : {index} label : {label}data : {data}')
    if data is None:
      entry = {
          cluster_key : [question]
      }
    else:
      data.append(question)
      entry = {
          cluster_key : data
      }
    clusters.update(entry)

  return clusters

In [111]:
clusters = get_clusters_content(labels)

In [56]:
# Create a Bokeh figure
p = figure(width=1800, height=600, title="Cluster Visualization")

# Define a colormap for clusters
mapper = linear_cmap(field_name='cluster_label', palette=Spectral11, low=min(labels), high=max(labels))

# Create a data source for Bokeh
source = ColumnDataSource(data={
    'x': reduced_vectors[:, 0],
    'y': reduced_vectors[:, 1],
    'cluster_label': labels,  # Replace with your cluster labels
    'questions' : qst,

})

# Create scatter plot glyphs
scatter = p.scatter(x='x', y='y', size=12, source=source, line_color=mapper, fill_color=mapper)
# Define the tooltip style with HTML/CSS attributes
tooltips = """
Question:

@questions

"""

# Add a hover tool with the defined tooltips
hover = HoverTool(renderers=[scatter], tooltips=tooltips)
p.add_tools(hover)

# Enable notebook display mode
# Customize the plot as needed (e.g., axis labels, legends, tooltips)
p.xaxis.axis_label = "X-axis"
p.yaxis.axis_label = "Y-axis"

output_notebook()
# Show the plot
show(p)


Loading BokehJS ...

In [66]:
import pandas as pd

In [123]:
clusters = sorted_dict = dict(sorted(clusters.items(),key=lambda item: len(item[1]),reverse=True))

In [124]:
for name,questions in clusters.items():
  print(name)
  for qst in questions:
    print('\t - ',qst)
  print('\n')

cluster_19
	 -  Can I choose my room preferences when I book?
	 -  Can I get assistance with tour bookings or tickets?
	 -  Can I arrange for a wake-up call in the morning?
	 -  Can I store valuable items in a hotel safe?
	 -  Can I change my room type upon arrival if available?
	 -  Can I arrange for early morning coffee or tea delivery?
	 -  Can I have flowers or gifts delivered to my room?
	 -  Can I use a hotel phone for local calls?
	 -  Can I upgrade my room after checking in?
	 -  Can I book a room for an extended period such as a month?
	 -  Can I request a late-night check-in or early check-out?
	 -  Can I get assistance with planning local sightseeing tours?
	 -  Can I book a conference room or business meeting space?
	 -  Can I have a private event with personal catering?
	 -  Can I receive detailed maps of local walking tours?
	 -  Can I have a personal trainer for in-room workouts?


cluster_3
	 -  What's your policy on early check-in requests?
	 -  What's the hotel's poli

### Note:
The next cell is about generating a markdown table to visualize the resulter clusterd in order to inculde it in the github repository Readme.md file.

In [125]:
first_10_clusters = {key: clusters[key] for key in list(clusters)[:10]}

In [131]:
def generate_clusters_markdown_table(clusters):
  markdown = "| Cluster name | Questions |\n| --- | ----------- |\n"
  for cluster_name,questions in clusters.items():
    qst = '<br>'.join(questions)
    markdown += f'| {cluster_name} | {qst} |\n'

  return markdown

In [129]:
result = {}
for i,(key,value) in enumerate(first_10_clusters.items(),start=1):
  result[f'{i}'] = value

In [132]:
print(generate_clusters_markdown_table(result))

| Cluster name | Questions |
| --- | ----------- |
| 1 | Can I choose my room preferences when I book?<br>Can I get assistance with tour bookings or tickets?<br>Can I arrange for a wake-up call in the morning?<br>Can I store valuable items in a hotel safe?<br>Can I change my room type upon arrival if available?<br>Can I arrange for early morning coffee or tea delivery?<br>Can I have flowers or gifts delivered to my room?<br>Can I use a hotel phone for local calls?<br>Can I upgrade my room after checking in?<br>Can I book a room for an extended period such as a month?<br>Can I request a late-night check-in or early check-out?<br>Can I get assistance with planning local sightseeing tours?<br>Can I book a conference room or business meeting space?<br>Can I have a private event with personal catering?<br>Can I receive detailed maps of local walking tours?<br>Can I have a personal trainer for in-room workouts? |
| 2 | What's your policy on early check-in requests?<br>What's the hotel's poli